In [1]:
import collections
import bisect
import glob
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats
import matplotlib.pyplot as plt
import matplotlib as mpl
import scipy.signal as signal

from sklearn.neighbors import KernelDensity
from sklearn.mixture import GaussianMixture

In [2]:
#plotting things

#%matplotlib qt5 -- I don't know what this is
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns

from cycler import cycler


#All of Anandh's customized seaborn/matplotlib settings

sns.set_context("talk", font_scale=1.5, rc={"lines.linewidth": 1.5})
sns.set_style("ticks")
sns.set_style({"xtick.direction": "in","ytick.direction": "in"})

#%config InlineBackend.figure_f.ormats=['svg']

mpl.rc('axes', prop_cycle=(cycler('color', ['r', 'k', 'b','g','y','m','c']) ))

mpl.rcParams['pdf.fonttype'] = 42
mpl.rcParams['ps.fonttype'] = 42

#mpl.rc('text', usetex=False)
#mpl.rc('text.latex', preamble=r'\usepackage{helvet}
#\renewcommand\familydefault{\sfdefault}\usepackage{sansmath}\sansmath')

    #If you want to use a different font
# mpl.rc('font',**{'family':'sans-serif','sans-serif':['Helvetica'], 
#                  'serif': ['Helvetica']})

tw = 1.5
sns.set_style({"xtick.major.size": 3, "ytick.major.size": 3,
               "xtick.minor.size": 2, "ytick.minor.size": 2,
               'axes.labelsize': 16, 'axes.titlesize': 16,
               'xtick.major.width': tw, 'xtick.minor.width': tw,
               'ytick.major.width': tw, 'ytick.minor.width': tw})

mpl.rc('xtick', labelsize=14) 
mpl.rc('ytick', labelsize=14)
mpl.rc('axes', linewidth=1.5)
mpl.rc('legend', fontsize=14)
mpl.rc('figure', figsize=(9,8))

In [3]:
def get_values(file_directory, channel='GFP/FITC-A', gate = None):
    '''
    Reads in the values from a specific channel for a given flow file.
    Defaults to taking GFP/FITC-A. If you want to get the values inside a certain gate,
    the gate argument is the STR name of the column with the gate 0/1 values; defaults to None
    '''
    flow_data = pd.read_csv(file_directory)
    
    #allowing us to use a gate we have assigned using other scripts
    #if no gate desired
    if gate==None:
        pass
    #if you give it a gate, subselect all the data where the gate == 1
    else:
        flow_data = flow_data.loc[flow_data[gate] == 1, :]
    
    #apply the log transform
    flow_data_channel_values = np.log10(flow_data[channel].values)
    
    return flow_data_channel_values

In [4]:
def make_df(file_directory, channel='GFP/FITC-A', gate = None):
    '''
    Creates a dataframe from the given file directory. Reads in all csvs, 
    extracts the data from the channel of interest (defaults to GFP/FITC-A), 
    and returns one dataframe. If you want to get the values inside a certain gate,
    the gate argument is the STR name of the column with the gate 0/1 values; defaults to None
    '''
    all_files = glob.glob(file_directory)
    all_files.sort()
    
    all_data = []
    for file in all_files:
        data = get_values(file, channel, gate)
        all_data.append(data)
    
    labels = []
    for i in range(0, len(all_files)):
        #this won't get the well label right in the output df unless the file format is:
        #'../../../..\'(letter)(number).csv'
        #the forward slash is from glob filenames. this can't handle 'gated_data' at the beginning of the filename
        mini_label = str(all_files[i].split('/')[-1].split('\\')[1].split('.')[0])
        #print(mini_label)
        label = [mini_label]*len(all_data[i])
        labels.append(label)
    
    flat_all_data = [item for sublist in all_data for item in sublist]
    flat_labels = [item for sublist in labels for item in sublist]
    
    df = pd.DataFrame(dict(well=flat_labels, log10values=flat_all_data))
    return df

In [5]:
def get_peak_locations_from_KDE_fit(data):
    ''' Performs a KDE fit and then uses scipy.signal.find_peaks_cwt to get peaks.
        The KDE bandwith parameter is critical, and 0.25 has worked well in the past.
        If it feels like you are missing many peak calls, decrease the bandwith. If it feels
        like you are having too many peak calls, increase the bandwith. 
        
        Don't change the bandwith without good reason, it took awhile to decide on 0.25. '''
    
    kde = KernelDensity(bandwidth=0.25, kernel='gaussian')

    kde.fit(data[:, None]);

    x_range = np.linspace(0, 6, 1200)
    kde_estimates = np.exp(kde.score_samples(x_range[:, None]))

    #Use the SciPy function to get the KDE peaks
    peaks = signal.find_peaks_cwt(kde_estimates, np.arange(30, 200), min_snr=1)

    means_init = []
    
    for peak in peaks:
        means_init.append(x_range[peak])
    
    return means_init

In [6]:
def fit_GMM_KDE(data, peaks, threshold = 0.01): 
    """Generate a Gaussian mixture model from the output
    of a Gaussian Kernel Density Estimation. 
    Outputs the mean of the on peak, fraction on, mean of the off peak, 
    and fraction off. This version of the code assumes all cells not in the on peak are off!
    This is obviously only a good assumption for uni/bimodal data. If you have multimodal data,
    do not use this code."""
    
    data = data.reshape(len(data), 1)

    peaks = np.array(peaks).reshape(len(peaks), 1)
    opt_gmm = GaussianMixture(n_components = len(peaks) , means_init = peaks).fit(data)  

    labels = opt_gmm.predict(data)
    labels = np.ravel(labels.reshape(len(labels), 1))

    means = opt_gmm.means_
    
    #this df contains each measurement value and the gaussian it is associated with.
    #you can use this to pull out the measurements that fall into the desired gaussian for gating.
    df = pd.DataFrame({'fluor value': np.ravel(data), 'which_gaussian': labels})


    #df.head(10)
    counts = []
    means = []



    for i in range(0, len(peaks)):
        df_distro = df.loc[df['which_gaussian']==i]
        counts.append(len(df_distro))
        means.append(np.mean(df_distro['fluor value'].values))

    print('peaks identified in the kde fit of data: ', peaks)
    print('means of the gaussians the GMM optimized: ', means)
    print('counts in each gaussian: ', counts)
    total = len(df)
    # print('total: ', total)

    fractions = np.array(counts)/total
    print('raw fractions of data in each gaussian: ', fractions)

    ##Initializing corrected lists of means and fractions of subpopulations
    GMM_accepted_means = []

    GMM_corrected_fractions = []

    for i in range(0, len(fractions)):
        if fractions[i] > threshold: 
            GMM_accepted_means.append(means[i])
            GMM_corrected_fractions.append(fractions[i])    

    print('means of gaussians that contain fraction of data above thresh ({}): '.format(threshold), GMM_accepted_means)
    print('fraction of data in above-thresh gaussians: ', GMM_corrected_fractions)

    #uses the gaussian with the highest mean (the last one in the accepted means) as the ON gaussian
    index_of_on = GMM_accepted_means.index(max(GMM_accepted_means))

    #the threshold collects all the fractions that are above threshold, meaning its possible for the sum
    #of the collected fractions to be less than 1. This is fine because this line tosses everything except
    #the fraction of measurements in the ON gaussian and calculates the fraction in the OFF as 1-this, 
    #saying that any sub-threshold counts above this are OFF and everything below this is also OFF
    fraction_of_highest_peak = GMM_corrected_fractions[index_of_on]

    fraction_off = 1 - fraction_of_highest_peak

#     print('fraction in highest gaussian: ', fraction_of_highest_peak)
#     print('1 - fraction_in_highest_gaussian: ', fraction_off)

    mean_of_highest_peak = GMM_accepted_means[index_of_on]

    #because there can be many possible above-threshold means that are OFF (below the highest mean-ed gaussian (ON) )
    #we want to come up with an appropriate summary statistic for their value. This can be their mean mean, weighted by
    #the fraction of the data they represent
    #This means we multiply the mean of each non-ON gaussian by the fraction of measurements in that gaussian, then
    #take the average of these weighted values and divide by the total fraction of data represented in all these 
    #gaussians to get the weighted average value for all the non-ON data.
    weighted_peak_means = []
    fracs_to_use = []
    for i in range(0, len(GMM_corrected_fractions)):
        #if its the one corresponding to the highest gaussian, do nothing
        if i == index_of_on:
            pass
        #otherwise, compute the weighted average using this gaussian's data too.
        else: 
            #we retransform out of log space because avg of 2 and 3 in log is 2.5,
            #but real average is scaled by log 10 and avg value is not 10**2.5
            weighted_mean = GMM_corrected_fractions[i] * np.power(10, GMM_accepted_means[i])
            fracs_to_use.append(GMM_corrected_fractions[i])
            weighted_peak_means.append(weighted_mean)
            

    mean_of_off_population = np.log10(np.sum(weighted_peak_means)/np.sum(fracs_to_use))

    return mean_of_highest_peak, fraction_of_highest_peak, mean_of_off_population, fraction_off, df

In [7]:
def fit_GMM_KDE_wrapper (data):
    """Wrapper function to get both the peaks from a KDE fit, and then 
    from the Gaussian mixture model. Returns the mean of the broken cells, 
    and the fraction of broken cells."""
    
    peak_locations = get_peak_locations_from_KDE_fit(data)
    
    mean_of_highest_peak, fraction_of_highest_peak, \
        mean_of_off_population, fraction_off, \
        df_gaussian_distros = fit_GMM_KDE(data, peak_locations, threshold = 0.01)
    
    return mean_of_highest_peak, fraction_of_highest_peak, mean_of_off_population, fraction_off, df_gaussian_distros

In [8]:
def GMM_method(df_, wells, gate = None):
    '''Wrapper function for the the entire generation of the final output df. 
    Takes the input dataframe and a list of all wells you want to perform GMM fitting on.
    Adds a column for gate used, which is a shared input variable for the data INPUT stuff
    and this data analysis stuff.
    '''
    means_on = []
    fractions_on = []
    means_off = []
    fractions_off = []
    wells_df = []
    dfs_gaussian_distros = []
    
    for well in wells:
        data = df_.loc[df_['well'] == well]
        
        vals = data['log10values'].values
        
        mean_of_highest_peak, fraction_of_highest_peak, mean_of_off_population, \
                        fraction_off, df_gaussian_distros = fit_GMM_KDE_wrapper(vals)
        
        means_on.append(mean_of_highest_peak)
        fractions_on.append(fraction_of_highest_peak)
        means_off.append(mean_of_off_population)
        fractions_off.append(fraction_off)
        dfs_gaussian_distros.append(df_gaussian_distros)
        wells_df.append(well) 

    plt_df = pd.DataFrame({'mean ON': means_on, 'fraction ON' : fractions_on,
                           'mean OFF':means_off, 'fraction OFF' : fractions_off,
                           'well': wells_df})#, 'df_gaussian_distros': dfs_gaussian_distros})
                                                #dont need ^^^ these anymore now that I've implemented GMM gating
                                                #somewhere else
    plt_df['gate'] = gate
    
    return plt_df

In [9]:
def run_GMM_frac_analysis (file_exp, channel, wells, gate = None):
    """
    Runs the whole GMM fractional analysis on a bunch of files, getting you location and fraction ON and OFF
    in a particular channel
    
    Takes:
    
    file_exp  :  regular expression that tells glob what files to run
    channel  :  the fluorescence channel data you want to analyze
    wells  :  the list of wells you want to analyze, allows analysis of subset of the files in file_exp
    gate  :  the gate inside which you want to do the analysis, DEF to None
    
    ------------
    Returns
    
    output_df  :  a dataframe that has mean locations ON/OFF, fractions of reads in the ON/OFF gaussians,
                    the corresponding well, the gate in which the analysis was applied to this well
    """
    
    #Read in the tidied data generated by any of your gating scripts
    # The input for make_df is the directory that you want automatic fractions generated for. 
    input_df = make_df(file_exp, channel=channel, gate=gate)
    
    input_df = input_df.dropna()
    
    #now we have our mature input data, channel selected and gated
    
    #check if it looks right
    print(input_df.head())
    
    ######## now do the analysis #########
    
    # Generate an output_df by running GMM method. The inputs are the input_df and
    # the list of all wells you want generated
    output_df = GMM_method(input_df, wells, gate=gate)
    
    return output_df

In [10]:
# Initialize the list of wells that you have data files for and wish to fit with GMMs

#all 96 wells
wells = [i+j for i in ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H'] for j in ['1','2','3', '4', '5', '6', '7', '8', '9', '10', '11', '12']]

In [11]:
#set some static variables

channels = {'syto' : 'mKate/APC-A', 'yfp' : 'GFP/FITC-A', 'bfp' : 'CFP/VioBlue-A', 'cfp' : 'VioGreen-A'}


In [38]:
#set important variables
upper_dir = 'Z:/Reed/Projects/micro_consortia/DARPA_biocon/Task 1.1/A=B/20190214 A=B mar cfp yfp small screen 1/flow/'
tpt = '18'
f = upper_dir + tpt + '/*.csv'

key = 'yfp'
c = channels[key]

w = wells

gt = ['mKate/APC-A_GMMgate']

In [40]:
all_analysis = []

for g in gt:
    add = run_GMM_frac_analysis(f, c, w, g)
    
    all_analysis.append(add)
    
analysis = pd.concat(all_analysis, axis='index')

C:\Users\geeze\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in log10


  well  log10values
0   A1     2.477515
1   A1     3.092553
2   A1     3.054189
3   A1     2.251818
4   A1     3.300649


C:\Users\geeze\Anaconda3\lib\site-packages\scipy\stats\stats.py:1713: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return np.add.reduce(sorted[indexer] * weights, axis=axis) / sumval
C:\Users\geeze\Anaconda3\lib\site-packages\ipykernel_launcher.py:94: RuntimeWarning: invalid value encountered in double_scalars


peaks identified in the kde fit of data:  [[2.87239366]]
means of the gaussians the GMM optimized:  [2.491311247539339]
counts in each gaussian:  [63830]
raw fractions of data in each gaussian:  [1.]
means of gaussians that contain fraction of data above thresh (0.01):  [2.491311247539339]
fraction of data in above-thresh gaussians:  [1.0]


C:\Users\geeze\Anaconda3\lib\site-packages\scipy\stats\stats.py:1713: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return np.add.reduce(sorted[indexer] * weights, axis=axis) / sumval


...

C:\Users\geeze\Anaconda3\lib\site-packages\scipy\stats\stats.py:1713: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return np.add.reduce(sorted[indexer] * weights, axis=axis) / sumval
C:\Users\geeze\Anaconda3\lib\site-packages\ipykernel_launcher.py:94: RuntimeWarning: invalid value encountered in double_scalars


peaks identified in the kde fit of data:  [[3.34778982]]
means of the gaussians the GMM optimized:  [3.0246343359083787]
counts in each gaussian:  [10075]
raw fractions of data in each gaussian:  [1.]
means of gaussians that contain fraction of data above thresh (0.01):  [3.0246343359083787]
fraction of data in above-thresh gaussians:  [1.0]


C:\Users\geeze\Anaconda3\lib\site-packages\scipy\stats\stats.py:1713: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return np.add.reduce(sorted[indexer] * weights, axis=axis) / sumval


peaks identified in the kde fit of data:  [[2.04170142]
 [3.89824854]]
means of the gaussians the GMM optimized:  [1.877306402924504, 3.7646019480092754]
counts in each gaussian:  [178, 6]
raw fractions of data in each gaussian:  [0.9673913 0.0326087]
means of gaussians that contain fraction of data above thresh (0.01):  [1.877306402924504, 3.7646019480092754]
fraction of data in above-thresh gaussians:  [0.967391304347826, 0.03260869565217391]
peaks identified in the kde fit of data:  [[2.07172644]
 [3.94328607]]
means of the gaussians the GMM optimized:  [1.9265822993642225, 3.8321278020858482]
counts in each gaussian:  [145, 11]
raw fractions of data in each gaussian:  [0.92948718 0.07051282]
means of gaussians that contain fraction of data above thresh (0.01):  [1.9265822993642225, 3.8321278020858482]
fraction of data in above-thresh gaussians:  [0.9294871794871795, 0.07051282051282051]


C:\Users\geeze\Anaconda3\lib\site-packages\scipy\stats\stats.py:1713: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return np.add.reduce(sorted[indexer] * weights, axis=axis) / sumval


In [41]:
analysis

,mean ON,fraction ON,mean OFF,fraction OFF,well,gate
0,2.491311,1.000000,NaN,0.000000,A1,mKate/APC-A_GMMgate
1,3.130415,0.330556,2.118793,0.669444,A2,mKate/APC-A_GMMgate
2,2.569426,1.000000,NaN,0.000000,A3,mKate/APC-A_GMMgate
3,2.353398,1.000000,NaN,0.000000,A4,mKate/APC-A_GMMgate
4,2.141832,0.924962,1.207607,0.075038,A5,mKate/APC-A_GMMgate
5,2.057833,1.000000,NaN,0.000000,A6,mKate/APC-A_GMMgate
6,2.784997,1.000000,NaN,0.000000,A7,mKate/APC-A_GMMgate
7,2.705298,1.000000,NaN,0.000000,A8,mKate/APC-A_GMMgate
8,2.771249,1.000000,NaN,0.000000,A9,mKate/APC-A_GMMgate
9,2.710597,1.000000,NaN,0.000000,A10,mKate/APC-A_GMMgate


In [42]:
# Write the output file to a csv. 
analysis.to_csv(upper_dir +'/' + tpt + '_' + key + '_GMManalysis.csv', index = False)